### Create document loader

Install specific dependencies and packages for document handling

In [ ]:
%pip install --q unstructured langchain
%pip install --q "unstructured[all-docs]"
%pip install --q langchain-community

In [ ]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader
import os
from dotenv import load_dotenv
load_dotenv()

### Load documents

In [ ]:
local_path = os.getenv("LOCAL_PATH")

# Local PDF file uploads
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

let's explore docs

In [ ]:
# data type must be document
type(data[0])

In [ ]:
# Preview first page
data[0].page_content

In [ ]:
# show with pretty format
data[0].page_content.split("\n\n") # select the correct splitter character

In [ ]:
# number of characters in raw document
print(f"N chars in raw doc: {len(data[0].page_content)}")

### RAG Configuration

download embedding model and chat model (mistral)

In [ ]:
!ollama pull nomic-embed-text
!ollama pull mistral
#!ollama list --uncomment this line for validate all models were downloaded

Install dependencies for local vector database and text splitters

In [ ]:
%pip install --q chromadb
%pip install --q langchain-text-splitters

In [ ]:
# load langchain dependencies for use ollama Embeddings and Chromadb
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [ ]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100) # select the properly chunk size 
chunks = text_splitter.split_documents(data)

In [ ]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

### Retrieve

In [ ]:
# dependencies for chatting and chaining
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
# LLM from Ollama
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [ ]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate three
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [ ]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke(input(""))

In [ ]:
chain.invoke("Qué dice la teoría del lugar central?")

In [ ]:
# Delete all collections in the db
vector_db.delete_collection()